In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
from glob import glob
import json
from collections import defaultdict
import numpy as np

In [ ]:
def get_probe_df(output_path, error_types, pooling, model_prefix, direction, base_model_name=None, clean=False):
    probe_metrics_dict = defaultdict(list)
    
    for error in error_types:
        model_name = f"{model_prefix}-{direction}-clean-{error}" if clean else f"{model_prefix}-{direction}-{error}"
        model_name = base_model_name if base_model_name else model_name
        
        for i, dir_path in enumerate(glob(f'{output_path}/{model_name}/*')):
            filename = f"{dir_path}/{pooling}/{model_prefix}.multi/best_epoch/test_metrics.json"
            if not Path(filename).exists():
                continue
            with open(filename, 'r') as f:
                metrics = json.load(f)
                probe_metrics_dict["Model"].append(model_name)
                probe_metrics_dict["Layer"].append(str(i + 1))
                probe_metrics_dict["Error"].append(error)
                for k, v in metrics.items():
                    probe_metrics_dict[k].append(v) 

    return pd.DataFrame(probe_metrics_dict)

In [ ]:
plt.rcParams['figure.dpi'] = 300
sns.set_theme(style='whitegrid', font_scale=1.4)

models = {
    'opus': 'opus-mt',
    'm2m100': 'm2m100_418M',
    'mbart': 'mbart-large-50-many-to-many-mmt', 
    'nllb': 'nllb-200-distilled-600M', 
}
error_types = ['morpheus']
figure_path = f'../results/figures/GED_Probing'
directions = ['en-es', 'fr-es', 'en-de', 'en-it', 'en-nl']

Path(figure_path).mkdir(exist_ok=True, parents=True)

combined_df_list = []
for direction in directions:
    
    for model_prefix, model_name in models.items():
        PROBING_OUTPUT_PATH = f'../outputs/probes/{direction}'
        
        finetuned_probe_df = get_probe_df(
            output_path=PROBING_OUTPUT_PATH, 
            error_types=error_types, 
            pooling='last', 
            model_prefix=model_prefix, 
            direction=direction
        )
        finetuned_probe_df['Model'] = 'Noise-Finetuned'
        finetuned_probe_df['Model Prefix'] = model_prefix.upper()

        clean_finetuned_probe_df = get_probe_df(
            output_path=PROBING_OUTPUT_PATH, 
            error_types=error_types, 
            pooling='last', 
            model_prefix=model_prefix, 
            direction=direction, 
            clean=True
        )
        clean_finetuned_probe_df['Model'] = 'Clean-Finetuned'
        clean_finetuned_probe_df['Model Prefix'] = model_prefix.upper()

        if 'opus' in model_name:
            model_name = f"{model_name}-{direction}"

        base_probe_df = get_probe_df(
            output_path=PROBING_OUTPUT_PATH, 
            error_types=error_types, 
            base_model_name=model_name, 
            pooling='last', 
            model_prefix=model_prefix, 
            direction=direction
        )
        base_probe_df['Model'] = 'Base'
        base_probe_df['Model Prefix'] = model_prefix.upper()

        # Combine DataFrames for the current model
        probe_df = pd.concat([clean_finetuned_probe_df, finetuned_probe_df, base_probe_df], axis=0)
        probe_df['Language'] = '-'.join([d.capitalize() for d in direction.split('-')])
        combined_df_list.append(probe_df)

combined_df = pd.concat(combined_df_list)
combined_df['Error'] = combined_df['Error'].apply(lambda x: x.capitalize())

g = sns.catplot(
    x='Layer', 
    y='binarized-f1', 
    hue='Model', 
    row='Model Prefix', 
    col='Language',        
    data=combined_df, 
    kind='point', 
    height=5, 
    aspect=1.2,
    palette='Set2',
    hue_order=['Noise-Finetuned', 'Clean-Finetuned', 'Base'],
    sharey=True  
)

for ax in g.axes.flat:
    ax.tick_params(axis='x', labelsize=20)
    ax.tick_params(axis='y', labelsize=20)

sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(0.5, -0.03), ncol=3, title=None, frameon=False,
    )

plt.ylim(0.0, 1.0)
plt.subplots_adjust(hspace=0.2, wspace=0.1) 

plt.setp(g._legend.get_texts(), fontsize=30)
g.fig.suptitle(f"GED Probing F1 Score: Morpheus", fontsize=36, y=1.05)
g.set_axis_labels("Layer", "F1 Score", fontsize=30)
g.set_titles(col_template="{col_name}", row_template="{row_name}", size=30)

plt.savefig(f'{figure_path}/ged_probing_morpheus.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{figure_path}/ged_probing_morpheus.pdf', dpi=300, bbox_inches='tight')

plt.show()
